# Part 1: Implementing a feedforward neural network using NumPy

We'll implement a neural network on the MNIST dataset -- from scratch only using the Numpy library. That will allow us to build an intuition of how our model actually works beyond the surface level PyTorch implementation that we'll do later in this project. 

Like I said, this will be a feedforward neural network, meaning information only flows forward during inference. To do this, we'll make use of dense layers, the ReLU activation function, categorical cross-entropy loss, and a softmax activation function for the output. I've made a little diagram of what we're going to be implementing below:
> INSERT IMG

I'll work on everything in sections, implementing it with example classes in this jupyter notebook, and adding the real classes that we'll use into the classes.py file in this same folder (filepath: fnn/p1-numpy/classes.py).

As a precursor to all our work, let's initialize the necessary packages in the workspace. 

In [1]:
# Package Initialization
import math
import numpy as np
from classes import DenseLayer, ReLU, Softmax, CategoricalCrossEntropyLoss

Now we can begin the implementation :)

Dense layers, otherwise known as fully-connected layers, are the foundation of neural networks. A dense layer is really quite simple, if you understand the workings of matrix multiplication. I'll briefly go over it, but if you need a better introduction to dense layers I would recommend the book *Neural Networks from Scratch* by Kinsley and Kukieła.

# Section 1: The Forward Pass of Dense Layers

At the core level, the way a neural network layer works is matrix multiplication. We can work through an example for this. Let's think of this as creating a function where we input a list X, of length 3 with integers, and it outputs a list Y, of length 4 with sum(X) in each entry. You can look at the diagram below for an example:

<br/>

<img src="../../diagrams/ex1.png" alt="A diagram showing our proposed 'function.'"/>

Skip this if you understand matrix multiplication: effectively, what we're doing is just a function that carries out matrix multiplication. Matrix multiplication is done by the element wise multiplication of two matrices of sizes (n x m) and (m x p), respectively. As you can see there, that middle dimension adding up is key, and that's because we're doing row x column. I won't go too into the weeds there, but let's look at a mini-example -- almost recursive if that makes sense :). Let's say we're trying to multiply a matrix X of size (1 x 2) times another matrix Y of size (2 x 2). That can be done because the middle dimension lines up. If matrix X is [a, b] and matrix Y is [[c, d],[e, f]]. That is, where every column of matrix Y represents the weights of one neuron as a column vector. Then, what matrix multiplication is doing is just [(a\*c + a\*e), (b\*d + b\*f)]. That's it for this recursive example, and I hope that makes sense.

Now, I'll quickly use Numpy to show the main example I'm referring to above. Please note, np.array() is the standard way for creating arrays or matrices in numpy. 

In [9]:
# Create the input array of size (1 x 3)
inputs = np.array([1., 2., 4.])
# Create the weight array of size (3 x 4) as we have 3 inputs and 4 desired outputs
weights = np.array([[1., 1., 1., 1.],
                    [1., 1., 1., 1.],
                    [1., 1., 1., 1.]])

# Perform matrix multiplication on the inputs and weights, note np.dot() is the standard and combined way of performing both dot products (which are normally for vectors) and matrix multiplication (which is for matrices)
outputs = np.dot(inputs, weights)

print(f"The matrix mult. output is: {outputs}")

The matrix mult. output is: [7. 7. 7. 7.]


That should about give you an intuition of how a neural network layer really works, but there's one more thing to address in this part of things: biases. 

Biases are parameters which are trainable and which are meant to offset the outputs positively or negatively. Each neuron has its own bias parameter, which we usually initialize biases to 0 and then modify according to the gradients during training. It's really quite a simple concept and there's not much more to it.

In practice, these biases are added as scalars (integers) to the product of each neuron. Let's look at how this happens, using our inputs and weights from the previous code cell.

In [12]:
# INPUTS AND WEIGHTS ARE PRE-LOADED FROM THE EARLIER CELL

# NEW: create the biases 
biases = np.array([1., 2., 3., 4.])

# Perform matrix multiplication on the inputs and weights, note np.dot() is the standard and combined way of performing both dot products (which are normally for vectors) and matrix multiplication (which is for matrices)
outputs = np.dot(inputs, weights)

print(f"The outputs before adding biases are: {outputs}")

outputs += biases

print(f"The outputs after adding biases are: {outputs}")

The outputs before adding biases are: [7. 7. 7. 7.]
The outputs after adding biases are: [ 8.  9. 10. 11.]


Now, we understand how the basics of how the whole thing works, at least in the forward pass. All in all, each neuron is really just carrying out ```y=mx+b``` where m is the input, x is the weight, b is the neuron's bias, and the result y is the neuron's output.

Great, so we can now implement the forward class of our dense layer. Like I said above, I'll implement an example class here but all the changes will be saved in the corresponding classes.py file.

In [17]:
class exDenseLayer:
    # Initialization method to initialize a dense layer object
    def __init__(self, numInputs, numNeurons):
        # Create random weights and scale them down, of size (inputs x neurons)
        self.weights = 0.01 * np.random.randn(numInputs, numNeurons)
        # Create an array filled with 0's with a bias for each neuron
        self.biases = np.zeros((1, numNeurons))
    
    # Our forward pass method
    def forward(self, inputs):
        # Calculate outputs using the method previously discussed
        self.outputs = np.dot(inputs, self.weights) + self.biases

That's about everything we need for a forward pass! Now we get to talk about the ReLU activation function!

## Section 2: The Various Activation Functions
Activation functions are necessary parts of neural networks if we ever want to model non-linear functions. In this section, we'll cover two of them: ReLU and Softmax.

### Section 2.1: The ReLU Activation Function

ReLU stands for "rectified linear unit" and is a piecewise activation function that makes the best of all worlds. It basically works using the following equation:
$$
f(x) = 
\begin{cases}
    x & \text{ if } x > 0 \\
    0 & \text{ otherwise}
\end{cases}
$$

Let's implement a class for our ReLU activation function, making use of the properties of Numpy to simplify our work.

In [5]:
class exReLU:
    # Our forward method; no overriding init method needed
    def forward(self, inputs):
        # Using the np.maximum() method to carry out our comparisons 
        self.output = np.maximum(0, inputs)

Now, let's try it out on an example array to see it in action. Spoiler, it's nothing crazy.

In [2]:
# Create a random array
exOutput = np.array([[-0.5, 0.3],
                   [0.43, -0.1]])

# Instantiate a ReLU object
activation = ReLU()

# Use the ReLU on our example outputs
activation.forward(exOutput)

print(activation.output)

[[0.   0.3 ]
 [0.43 0.  ]]


As we can see, the values less than 0 have been zerod out, whereas all the non-negative values have remained as they were!

And that's all there is to ReLU. It basically zeros out values less than 0 and keeps values greater than or equal to 0. Now we can move to the second activation function we'll look at: Softmax.

### Section 2.2: Softmax

The Softmax activation function is the one that we'll be using on our outputs. The Softmax is used in situations where we're to perform some kind of categorical operations. It works by taking the outputs and creating probabilities for each output class that ultimately add up to one -- giving us the "most likely" class.

The function for the Softmax function is:
$$
\frac{e^{z_{i,j}}}{\sum_{l=1}^{L} e^{z_{i,l}}}
$$

All that really means is it's getting the normalized probabilities for each, with it all adding up to 1.

So, let's implement this in a class. 

In [3]:
class exSoftmax:
    # Method for the forward pass; no init overriding needed
    def forward(self, inputs):
        # Calculate the non-normalized probabilities using np.exp(), which exponentiates everything, np.max() which finds the maximum value in the matrix, and np.sum() which performs a summation of each row/col as desired
        unProb = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        # Normalize the inputs for every sample
        nProb = unProb / np.sum(unProb, axis=1, keepdims=True)
        
        self.output = nProb

With that implemented, let's demonstrate how it works! 

In [5]:
# Create an array with hypothetical outputs
exOutput = np.array([[1, 4, 5]])

# Instantiate our Softmax object
activation = Softmax()

# Run our output through a Softmax layer
activation.forward(exOutput)

print(f"The normalized output probabilities are: {activation.output}")

The normalized output probabilities are: [[0.01321289 0.26538793 0.72139918]]


Awesome, and that's really it for our Softmax. There's not much more we need to know for the purposes of this implementation. Now, we're going to move to our method of measuring accuracy: the categorical cross-entropy loss.

## Section 3: Categorical Cross-Entropy Loss

Categorical cross entropy loss is a loss function which is used to compare a ground-truth probability (y, or targets) with some predicted distribution (y-hat, or predictions). It's one of the most commonly used loss functions with a softmax on the output, because it allows us to perform classification.

The formula for calculating the categorical cross-entropy is:
$$
L_{i} = - log(\hat{y}_{i,k})
$$
That is, where i denotes the sample loss value, i is the i-th sample in the set, k is the index of the target label, y denotes the target values, and y-hat denotes the predicted values.

Generally, we apply a log loss kind of function to the outputs of a binary log regression. That means, one where there are only two classes in the distribution. We apply this in our example where the target output is one-hot encoded, meaning there is only 0 or 1.

So, let's create our class to do this!

In [2]:
class exCategoricalCrossEntropyLoss:
    # The forward method because we don't need to override any initializations
    def forward(self, yPred, yTrue):
        # Number of samples in the batch
        samples = len(yPred)
        # Clip our data from both ends to prevent division by 0
        yPred = np.clip(yPred, 1e-7, 1-1e-7)
        
        # Probabilities for one-hot encoded target values
        confidences = np.sum(yPred * yTrue, axis=1)
        # Losses
        negLogLikelihood = -np.log(confidences)
        
        return negLogLikelihood
        
    # A calculate method
    def calculate(self, output, y):
        # Calculate the loss
        sampleLoss = self.forward(output, y)
        # Calculate the mean loss
        dataLoss = np.mean(sampleLoss)
        
        return dataLoss

That's all we need for our categorical cross entropy class. Let's test it out to see how it works!

In [12]:
# Initialize the loss object
lossFunction = CategoricalCrossEntropyLoss()

# Test one, totally accurate predictions.
aPred = np.array([[0, 1, 0],
                  [1, 0, 0]])
aTrue = np.array([[0, 1, 0],
                  [1, 0, 0]])
print(f"The total loss is {lossFunction.calculate(aPred, aTrue):.3f}, which should be 0.")

# Test two, medium but correct accuracy predictions.
aPred = np.array([[.4, .6, 0],
                  [.2, .7, .1]])
aTrue = np.array([[0, 1, 0],
                  [0, 1, 0]])
print(f"The total loss is {lossFunction.calculate(aPred, aTrue):.3f}, which should be nonzero.")

# Test three, low but correct accuracy predictions.
aPred = np.array([[.4, .3, .3],
                  [.33, .34, .33]])
aTrue = np.array([[1, 0, 0],
                  [0, 1, 0]])
print(f"The total loss is {lossFunction.calculate(aPred, aTrue):.3f}, which should be nonzero.")

# Test four, totally wrong predictions.
aPred = np.array([[1, 0, 0],
                  [0, 1, 0]])
aTrue = np.array([[0, 0, 1],
                  [1, 0, 0]])
print(f"The total loss is {lossFunction.calculate(aPred, aTrue):.3f}, which should be very high.")


The total loss is 0.000, which should be 0.
The total loss is 0.434, which should be nonzero.
The total loss is 0.998, which should be nonzero.
The total loss is 16.118, which should be very high.


As you can see, there's a very low loss (0) when the model makes totally accurate predictions, which begins to go up as predictions get less accurate. That's about all we need to know.

## Section 4: Backpropagation & Gradients

I've waited to introduce all the underlying concepts so that we can work through the idea of backpropagation with a complete picture of how a forward pass really works. 

From the surface level, backpropagation may seem to be the most complex part of our neural network -- but I'll try to do my best to explain it succinctly. 

At the core of backpropagation lies the principle of the chain rule, from calculus. First, let's give an example of how a single neuron sums up its inputs and applies an activation function, from which we can then have an intuition of how the chain rule applies.

<br/>

<img src="../../diagrams/ex2.png" alt="A diagram showing our proposed 'function.'"/>

In the above image, it shows how a neuron would process inputs and apply the ReLU activation function to them. It's pretty simple, and the red numbers are values passed through the system, whereas black numbers are parameters.

When we look at it from right to left, we're using the outputs of previous actions to complete the next step. So, what that means here is: 
$$
ReLU(sum(i_{0} \times w_{0}, i_{1} \times w_{1}, i_{2} \times w_{2}, i_{3} \times w_{3}, bias))
$$
That is, where $i_{x}$ is the x'th input and $w_{x}$ is the x'th weight.

Therefore, for us to find the derivative of this, we need to apply the chain rule, which states that:
$$
\frac{d}{dx}[f(g(x))] = f'(g(x)) \times g'(x)
$$ 
We keep applying this chain rule recursively to find the gradients -- which are vectors of partial derivatives with respect to each variable.

I won't dive much deeper into partial derivatives, but I would recommend that you pick up a book like Neural Networks from Scratch by Kinsley and Kukieła to learn more about it :). Now, we need to implement the backward pass methods for each component!

Let's go back in the same order which we implemented things initially.
